
# Control general para el UNDCMotor y el UNThermal

En este notebook vamos a mostrar como implementar diferentes controladores para el UNThermal y el UNDCMotor


## Configuración


### Instalación de la libreria unmotor

Descomentar y ejecutar esta celda solo para instalar por primera vez o actualizar la libreria. __Asegúrese de instalar  [Git](https://git-scm.com/download/win/ "Git").__



In [ ]:
import subprocess
command = ["pip", "install", "-I", "--user", "git+https://github.com/nebisman/UNDCMotor.git@main#subdirectory=code/python_code"]
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
for line in process.stdout:
    print(line.strip())


### Instalación de la libreria unthermal

Descomentar y ejecutar esta celda solo para instalar por primera vez o actualizar la libreria. __Asegúrese de instalar  [Git](https://git-scm.com/download/win/ "Git").__



In [ ]:
import subprocess
command = ["pip", "install", "-I","--user", "git+https://github.com/nebisman/UNThermal.git@main#subdirectory=code/python_code"]
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
for line in process.stdout:
    print(line.strip())


### Importación de comandos de la libreria unmotor y de Matplotlib 

A continuación importamos la libreria unmotor, la libreria unthermal y definimos la función para simular en lazo cerrado. 

In [ ]:
# importacion de graficos y funciones interactivas
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# importación de las librerias unmotor, unthermal y control
import unmotor as mot
import unthermal as ter
import control as ct


def simulate_closed_loop(G, T, t0, t1, r0, r1):
    """
    This function simulates and plots the expected response according to the obtained model.
    Parameters:
       G: open-loop system transfer function
       T: closed-loop function
       r0: initial reference value
       r1: final reference value
       t0: time of the initial reference value
       t1: duration of the final reference value     
    """

    #compute the control signal
    Gur = ct.minreal(T/G)
    t = np.linspace(0, t0 + t1, 200)
    r = []
    for tc in t:
        if tc <= t0:
            r.append(0)
        else:
            r.append(r1-r0)
    r = np.array(r)
    tsim, ysim  = ct.forced_response(T, t, r)
    tsim, usim  = ct.forced_response(Gur, t, r)
    fig = plt.gcf()
    ay, au = fig.get_axes()
    ay.plot(tsim, ysim + r0, color = "#ff6600ff", alpha= 0.6)
    au.plot(tsim, usim, color ="#37c8abff", alpha= 1)
    ay.text(0, r1,  'Simulated output', fontsize=12, color='#ff6600ff')    
    fig.canvas.draw()
    return 



---


## Control general del UNDCmotor en velocidad

Vamos a hacer un controlador de velocidad para el motor a la velocidad de $500^o/s$. Primero definimos el sistema.


In [ ]:
my_dcmotor =  mot.MotorSystemIoT(plant_number = "xxxx", broker_address = "192.168.1.100");  

###  Programación de un controlador general de velocidad en la planta 
Vamos a programar y probar el siguiente controlador
$$C(s)=  \frac{0.0030844\,(s+43.95)\,(s+1.5)}{(s+5.119)\,(s+0.0004013)}$$

In [ ]:
# Aqui definimos la s de Laplace
s = ct.TransferFunction.s

# Aqui definimos la función de transferencia identificada para el sistema
Gvel =  357.8/(0.205*s + 1)

# Aqui definimos el controlador, por medio de la s de Laplace
C =  0.0030844* (s+43.95)* (s+1.5)/((s+5.119) * (s+0.0004013))

# Aqui definimos la función de transferencia de lazo cerrado con el controlador diseñado para simular
T = ct.feedback(C*Gvel,1) 

#aqui programamos el controlador C, con salida de velocidad
mot.set_controller(my_dcmotor, C, output='speed')

#Aqui hacemos nuestro experimento y simulamos
r0 = 0     # valor inicial de la referencia
r1 = 500   # valor final de la referencia
t0 = 0.5   # tiempo que dura en r0 
t1 = 2     # tiempo que dura en r1
mot.step_closed(my_dcmotor, t0=t0,t1=t1, r0=r0,r1=r1);
simulate_closed_loop(Gvel, T, t0, t1, r0, r1)

## Control general del UNDCMotor en posición
A continuación vamos a hacer un control general del UNDCMotor en posición. El controlador que vamos a implementar es el siguiente:

$$C_{pos}(s)= \frac{0.039597 (s+43.11) (s+4.222) (s+3.333) (s^2 + 21.02s + 256)}{s(s+19.05)(s+11.2)(s^2 + 22.81s + 544.2)}$$

Note que en Python $s^2$ se escribe como `s**2`


In [ ]:
# Aqui definimos la función de transferencia identificada para el sistema
Gpos =  357.8/(s*(0.205*s + 1))

# Aqui definimos el controlador, por medio de la s de Laplace

Cpos = 0.039597*(s+43.11)*(s+4.222)*(s+3.333)*(s**2 + 21.02*s + 256)/(s* (s+19.05)* (s+11.2) *(s**2 + 22.81*s + 544.2))

# Aqui definimos la función de transferencia de lazo cerrado con el controlador diseñado para simular
Tpos = ct.feedback(Cpos*Gpos,1) 

# Aqui programamos el controlador  con salida en angulo y compensación de la zona muerta, si es necesario
mot.set_controller(my_dcmotor, Cpos, 'angle', deadzone = 0.32)

# Aqui hacemos nuestro experimento y simulamos
r0 = 0     # valor inicial de la referencia
r1 = 90   # valor final de la referencia
t0 = 0.5   # tiempo que dura en r0 
t1 = 4     # tiempo que dura en r1
mot.step_closed(my_dcmotor, t0=t0,t1=t1, r0=r0,r1=r1);
simulate_closed_loop(Gpos, Tpos, t0, t1, r0, r1)

## Control general del UNThermal

A continuación vamos a programar un controlador general en el UNThermal. Primero definimos el sistema en el IoT. 

In [ ]:
my_thermal = ter.ThermalSystemIoT(plant_number = "XXXX", broker_address = "192.168.1.100");  

### Programación de un controlador general en el UNThermal

A continuación vamos a programar el siguiente controlador en el UNThermal

$$C_{ter}(s) =  \frac{6.681(s+1.016)(s+0.05)}{(s+0.246)(s+0.01394)}$$

In [ ]:
# Aqui definimos la función de transferencia identificada para el sistema
G_ter =   1.28/(34.7*s + 1)

# Aqui definimos el controlador, por medio de la s de Laplace
C_ter =    6.681*(s+1.016)*(s+0.05)/((s+0.246)*(s+0.01394))

# Aqui definimos la función de transferencia de lazo cerrado con el controlador diseñado para simular
T_ter = ct.feedback(C_ter * G_ter,1) 

#aqui programamos el controlador C para la planta termica

ter.set_controller(my_thermal, C_ter)

# aqui probamos y programamos nuestro controlador
r0 = 50     # valor inicial de la referencia
r1 = 60     # valor final de la referencia
t0 = 50     # tiempo que dura en r0 
t1 = 50     # tiempo que dura en r1
ter.set_controller(my_thermal, C_ter)
ter.step_closed(my_thermal, t0=t0,t1=t1, r0=r0,r1=r1);
simulate_closed_loop(Gvel, T_ter, t0, t1, r0, r1)